In [2]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import text
from utils import connection as con #se puede traer un aechivo por el __init

In [3]:
conn_retail_db = con.connect_retail_db()
conn_dw_retail = con.connect_dw_retail()

In [4]:
from etl import extract as ex

In [5]:
df_customers = ex.extract_table("customers", conn_retail_db)
df_orders = ex.extract_table("orders", conn_retail_db)
df_order_items = ex.extract_table("order_items", conn_retail_db)
df_products = ex.extract_table("products", conn_retail_db)
df_categories = ex.extract_table("categories", conn_retail_db)
df_departments = ex.extract_table("departments", conn_retail_db)


In [5]:
from etl import load

In [2]:
#load.load_dw_retail('dimension_customer', conn_dw_retail, df_customers.drop('customers_password', axis=1))

In [7]:
df = pd.read_sql("SELECT * FROM fact_orders", con=conn_dw_retail)
df

,order_id,order_date,order_customer_id,order_status,total_items,total_amount,time_id
0,1,2013-07-25,11599,CLOSED,1,299.98,20130725
1,2,2013-07-25,256,PENDING_PAYMENT,7,579.98,20130725
2,4,2013-07-25,8827,CLOSED,14,699.85,20130725
3,5,2013-07-25,11318,COMPLETE,10,1129.86,20130725
4,7,2013-07-25,4530,COMPLETE,7,579.92,20130725
...,...,...,...,...,...,...,...
57426,68879,2014-07-09,778,COMPLETE,3,1259.97,20140709
57427,68880,2014-07-13,1117,COMPLETE,17,999.77,20140713
57428,68881,2014-07-19,2518,PENDING_PAYMENT,1,129.99,20140719
57429,68882,2014-07-22,10000,ON_HOLD,2,109.99,20140722


In [17]:
load.load_dw_retail('dimension_order', conn_dw_retail, df_orders)

In [22]:
load.load_dw_retail('dimension_order_items', conn_dw_retail, df_order_items)

In [ ]:
from etl import transform as t

: 

In [ ]:
#No se puede hacer pot python asi que se ejcuta directamente el query en sql
query = """
SELECT DISTINCT DATE_FORMAT(order_date, '%Y%m%d') AS time_id,
YEAR(order_date) AS year,
QUARTER(order_date) AS quarter,
MONTH(order_date) AS month,
DAY(order_date) AS day,
DAYOFWEEK(order_date) AS day_of_week,
DAYOFYEAR(order_date) AS day_of_year,
WEEK(order_date) AS week_of_year,
CASE WHEN DAYOFWEEK(order_date) IN (1,7) THEN TRUE ELSE FALSE END AS is_weekend
FROM orders
"""

df_dimension_time = t.transform_time_from_order(query, conn_retail_db)
df_dimension_time.head()